##**Notebook PC#11**

## Encoder-Decoder LSTM for Natural Language Processing.

**Professor:** Fernando J. Von Zuben <br>
**Aluno(a):** Taylon Luan Congio Martins RA: 177379<br>
**Aluno(a):** Tiago C A Amorim RA: 100675

In [ ]:
from random import seed
from random import randint
from numpy import array
from numpy import argmax

In [ ]:
def random_sum_pairs(n_examples, n_numbers, largest):
    X,y = list(), list()
    for i in range(n_examples):
        in_pattern=[randint(1,largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X,y

In [ ]:
seed(1)
n_samples =1
n_numbers = 2
largest = 10
X,y = random_sum_pairs(n_samples, n_numbers, largest)
print(X,y)

[[3, 10]] [13]


In [ ]:
from math import ceil
from math import log10

In [ ]:
def to_string(X,y,n_numbers,largest):
    max_length = n_numbers*ceil(log10(largest+1)) + n_numbers - 1
    Xstr = list()
    for pattern in X:
        strp = '+'.join([str(n) for n in pattern])
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        Xstr.append(strp)
    maxlength = ceil(log10(n_numbers*(largest+1)))
    ystr = list()
    for pattern in y:
        strp = str(pattern)
        strp = ''.join([' 'for _ in range(maxlength-len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

In [ ]:
seed(1)
n_samples = 1
n_numbers = 2
largest = 10

In [ ]:
X,y = random_sum_pairs(n_samples, n_numbers, largest)
print(X,y)

X,y = to_string(X,y,n_numbers,largest)
print(X,y)

[[3, 10]] [13]
[' 3+10'] ['13']


In [ ]:
alphabet = ['0','1','2','3','4','5','6','7','8','9','+',' ']

In [ ]:
def integer_encode(X,y,alphabet):
    char_to_int = dict((c,i) for i,c in enumerate(alphabet))
    Xenc = list()
    for pattern in X:
        integer_encoded = [char_to_int[char] for char in pattern]
        Xenc.append(integer_encoded)
    yenc = list()
    for pattern in y:
        integer_encoded = [char_to_int[char] for char in pattern]
        yenc.append(integer_encoded)
    return Xenc, yenc

In [ ]:
X,y = integer_encode(X,y,alphabet)

In [ ]:
print(X,y)

[[11, 3, 10, 1, 0]] [[1, 3]]


In [ ]:
def one_hot_encode(X,y,max_int):
    Xenc = list()
    for seq in X:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)

    yenc = list()
    for seq in y:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

In [ ]:
X,y = one_hot_encode(X,y,len(alphabet))

In [ ]:
print(X,y)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]] [[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]]


In [ ]:
def generate_data(n_samples,n_numbers, largest, alphabet):
    X,y = random_sum_pairs(n_samples,n_numbers,largest)
    X,y = to_string(X,y,n_numbers,largest)
    X,y = integer_encode(X,y,alphabet)
    X,y = one_hot_encode(X,y,len(alphabet))
    X,y = array(X), array(y)
    return X,y

In [ ]:
def invert(seq,alphabet):
    int_to_char = dict((i,c) for i,c in enumerate(alphabet))
    strings  = list()
    for pattern in seq:
        string = int_to_char[argmax(pattern)]
        strings.append(string)
    return ''.join(strings)

In [ ]:
n_terms = 3
largest = 10
alphabet = [str(x) for x in range(10)] + ['+', ' ']

In [ ]:
n_chars = len(alphabet)
n_in_seq_length = n_terms*ceil(log10(largest+1)) +n_terms-1
n_out_seq_length = ceil(log10(n_terms*(largest+1)))

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Dense

In [ ]:
model = Sequential()
model.add(LSTM(75, input_shape=(n_in_seq_length,n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(50,return_sequences=True))
model.add(TimeDistributed(Dense(n_chars,activation='softmax')))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 75)                26400     
                                                                 
 repeat_vector (RepeatVecto  (None, 2, 75)             0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 2, 50)             25200     
                                                                 
 time_distributed (TimeDist  (None, 2, 12)             612       
 ributed)                                                        
                                                                 
Total params: 52212 (203.95 KB)
Trainable params: 52212 (203.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
X,y = generate_data(75000,n_terms,largest,alphabet)
model.fit(X,y,epochs=1,batch_size=10)

7500/7500 [==============================] - 55s 6ms/step - loss: 0.4088 - accuracy: 0.8780


In [ ]:
X,y = generate_data(100,n_terms,largest,alphabet)
loss,acc = model.evaluate(X,y,verbose=0)
print('Loss: %f, Accuracy: %f' %(loss,acc*100))

Loss: 0.032466, Accuracy: 100.000000


In [ ]:
for _ in range(10):
    X,y = generate_data(1,n_terms,largest,alphabet)
    yhat = model.predict(X,verbose=0)
    in_seq = invert(X[0],alphabet)
    out_seq = invert(y[0],alphabet)
    predicted = invert(yhat[0],alphabet)
    print('%s = %s (expect %s)' %(in_seq,predicted,out_seq))

   3+7+5 = 15 (expect 15)
  2+10+6 = 18 (expect 18)
   4+5+9 = 18 (expect 18)
   2+2+4 =  8 (expect  8)
 10+7+10 = 27 (expect 27)
  10+2+3 = 15 (expect 15)
   8+4+1 = 13 (expect 13)
   8+1+3 = 12 (expect 12)
   5+9+6 = 20 (expect 20)
   1+5+8 = 14 (expect 14)


<font color="green">
Atividade (a) <br>
Como são gerados os dados de treinamento?
</font>

**Resposta:**<br>
Os dados são gerados a partir de exemplos aleatórios de somas de inteiros, que são convertidos a sequências de elementos de um dicionário one-hot. É utilizada a função **generate_data()**, a qual realiza as seguintes transformações sequenciais nos dados a serem gerados:

**random_sum_pairs()**: cria um dataset de pares input-output, onde cada input é uma lista de inteiros aleatórios e cada output é a soma dos inteiros correspondentes.

**to_string()**: converte os dados numéricos criados em dados formatados como string.

**integer_encode()**: converte a string de caracteres em uma lista de inteiros baseados num alfabeto passado como parâmetro, no caso, o alfabeto utilizado é composto por: [“0”, “1”, “2”, “3”, “4”, “5”, “6”, “7”, “8”, “9”, “+”, “ “].

**one_hot_encode()**: converte a sequência de inteiros em uma representação one-hot-encoding que são vetores de dimensão do alfabeto (1x12) onde cada caractere é representado por um conjunto de zeros e um único valor 1.

**array()**: por fim, os dados de treinamento input-output são transformados em formato array da biblioteca numpy para serem ingeridos pelo modelo de aprendizado de máquina.

<font color="green">
Atividade (b) <br>
Como uma calculadora simples pode operar baseada no conceito de tradução de frases, ou seja, sem realizar operações algébricas?
</font>

**Resposta:**<br>
Utilizando o conceito de NLP, como neste notebook, é possível criar um modelo seq2seq onde dado um token (e por fim uma sequência de tokens) que pertence a um dicionário definido a priori, o modelo gera uma saída esperada, também sequencial, este é o princípio básico da operação de alguns tradutores. Uma arquitetura com bloco LSTM, cuja arquitetura é recorrente e possui dinâmica não-linear, consegue capturar dependências sequenciais dos dados de entrada por processarem os token de maneira sequencial e atualizarem seus estados internos baseados no tokens anteriores, isto ocorre porque no treinamento a máquina aprende a mapear sequências de entrada para sequências de saída. No exemplo, o dicionário é composto de números inteiros (“0...9”), acrescido do símbolo de operação de soma (“+”) e de espaço (“ “). Portanto, quando entramos com uma sequência como 3+7+5 a máquina consegue inferir a o valor 15, funcionando como uma calculadora.

Este modelo tem algumas limitações ao usar o one-hot-encoding como: alta dimensão e esparcialidade dos dados que não capturam semântica entre tokens, falta de contexto no embedding dos tokens.  

Para um dicionário pequeno (igual ao do notebook), este modelo e arquitetura são suficientes para bons resultados. Para uma calculadora mais complexa, com mais operações, pode-se ter que recorrer a arquiteturas mais avançadas como transformers, que possui mecanismo de self-attention e conseguem gerar um word-embedding que guarde relações semânticas e de contexto entre tokens.



A estrutura da rede foi desenhada para que consiga gerar uma sequência de saída em função de uma sequência de entrada. O que se espera da rede é que consiga inferir as relações semânticas entre os elementos que constituem cada sequência de entrada, como ignorar espaços no início e juntar dígitos para formar um número.

Em uma rápida análise, é possível que a rede tenha ‘decorado’ todas possíveis sequências. A base de dados foi construída por 75000 sequências, mas temos um total de 10^3=1000 possíveis sequências para o problema colocado. Desta forma, várias sequências se repetem na base de dados montada, e é grande a probabilidade de que todas as possíveis combinações fazem parte da base de dados. Aumentando o maior inteiro possível o problema fica bem mais difícil para a rede resolver, e a acurácia cai substancialmente:

Máximo=50, dados de treino ~50% do universo, acurácia de teste=70%.

Máximo=99, dados de treino ~7,5% do universo, acurácia de teste=60%.

Mesmo errando o valor da soma, a rede estima uma resposta próxima do valor real. Ou seja, a rede consegue representar as distâncias entre as sequências de saída como distâncias entre os inteiros que representam.